![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/05.5.Improved_Entity_Resolvers_in_SparkNLP_with_sBert.ipynb)

# Improved Entity Resolvers in Spark NLP with sBert

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs==5.3.4

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=True
nlp.install(refresh_install=True)

In [ ]:
from johnsnowlabs import nlp, medical

# Automatically load license data and start a session with all jars user has access to
params = {"spark.driver.memory":"24G", # Amount of memory to use for the driver process, i.e. where SparkContext is initialized
          "spark.kryoserializer.buffer.max":"2000M", # Maximum allowable size of Kryo serialization buffer, in MiB unless otherwise specified.
          "spark.driver.maxResultSize":"2000M"}
spark = nlp.start(spark_conf = params)
spark

👌 Detected license file /content/5.3.1.spark_nlp_for_healthcare.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.3.1, 💊Spark-Healthcare==5.3.1, running on ⚡ PySpark==3.4.0


<h1>!!! Warning !!!</h1>

**If you get an error related to Java port not found 55, it is probably because that the Colab memory cannot handle the model and the Spark session died. In that case, try on a larger machine or restart the kernel at the top and then come back here and rerun.**

## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

## Utility functions

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='icd10cm_code', hcc=False, aux_label=False):

    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []
    all_k_aux_labels=[]

    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):

        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        codes.append(code.result)
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))
        all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))

        if hcc:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])

        elif aux_label:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
        else:
            all_k_aux_labels.append([])

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes,'all_codes':all_codes,
                       'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})

    if hcc:

        df['billable'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[0] for i in x])
        df['hcc_status'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[1] for i in x])
        df['hcc_score'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[2] for i in x])
        df = df.drop(['all_k_aux_labels'], axis=1)

    elif not aux_label:

        df = df.drop(['all_k_aux_labels'], axis=1)

    return df



## ICD10CM pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
    .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
    .setInputCols(["ner_chunk"])\
    .setOutputCol("sbert_embeddings")\
    .setCaseSensitive(False)

icd10_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
    .setInputCols(["sbert_embeddings"]) \
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")

icd_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        icd10_resolver])

icd_lp = nlp.LightPipeline(icd_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
[OK!]


In [ ]:
text = 'bladder cancer'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 41.5 ms, sys: 10.1 ms, total: 51.7 ms
Wall time: 1.37 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,C67.9,"[C67.9, Z12.6, D09.0, D49.4, C79.11]","[bladder cancer [malignant neoplasm of bladder, unspecified], suspected bladder cancer [encounter for screening for malignant neoplasm of bladder], cancer in situ of urinary bladder [carcinoma in situ of bladder], tumor of bladder neck [neoplasm of unspecified behavior of bladder], malignant tumour of bladder neck [secondary malignant neoplasm of bladder]]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]"


In [ ]:
text = 'polydipsia'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 32.7 ms, sys: 15.9 ms, total: 48.6 ms
Wall time: 1.19 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,polydipsia,0,9,R63.1,"[R63.1, F63.89, E23.2, F63.9, O40, G47.5, M79.89, R63.2, R06.1, H53.8, I44.2, G47.9, G47.8, G47.1, Q30.8, I45.9, R00.2, R06.81, G47.54, R06.8, M35.3]","[polydipsia [polydipsia], psychogenic polydipsia [other impulse disorders], primary polydipsia [diabetes insipidus], psychogenic polydipsia (disorder) [impulse disorder, unspecified], polyhydramnios [polyhydramnios], parasomnia [parasomnia], polyalgia [other specified soft tissue disorders], polyphagia [polyphagia], biphasic stridor [stridor], oscillopsia [other visual disturbances], parasystole (disorder) [atrioventricular block, complete], dyssomnia [sleep disorder, unspecified], parasomnia (disorder) [other sleep disorders], hypersomnia (disorder) [hypersomnia], polyrrhinia [other congenital malformations of nose], palpitations - irregular [conduction disorder, unspecified], intermittent palpitations [palpitations], vagal apnea [apnea, not elsewhere classified], secondary parasomnia [parasomnia in conditions classified elsewhere], vagal apnoea [other abnormalities of breathing], polymyalgia [polymyalgia rheumatica]]","[0.0000, 0.0455, 0.0635, 0.0786, 0.1474, 0.1520, 0.1553, 0.1573, 0.1641, 0.1675, 0.1648, 0.1600, 0.1662, 0.1667, 0.1727, 0.1702, 0.1687, 0.1672, 0.1730, 0.1700, 0.1729]"


## ICD10CM-HCC pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

hcc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_hcc_code")\
      .setDistanceFunction("EUCLIDEAN")

hcc_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hcc_resolver])

hcc_lp = nlp.LightPipeline(hcc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[OK!]


In [ ]:
text = 'bladder cancer'

%time get_codes (hcc_lp, text, vocab='icd10cm_hcc_code', hcc=True)

CPU times: user 46.7 ms, sys: 7.96 ms, total: 54.7 ms
Wall time: 1.28 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,bladder cancer,0,13,C67.9,"[C67.9, Z12.6, D09.0, D49.4, C79.11]","[bladder cancer [malignant neoplasm of bladder, unspecified], suspected bladder cancer [encounter for screening for malignant neoplasm of bladder], cancer in situ of urinary bladder [carcinoma in situ of bladder], tumor of bladder neck [neoplasm of unspecified behavior of bladder], malignant tumour of bladder neck [secondary malignant neoplasm of bladder]]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]","[1, 1, 1, 1, 1]","[1, 0, 0, 0, 1]","[11, 0, 0, 0, 8]"


In [ ]:
text = 'polydipsia'

%time get_codes (hcc_lp, text, vocab='icd10cm_hcc_code', hcc=True)

CPU times: user 37.8 ms, sys: 11.4 ms, total: 49.2 ms
Wall time: 1.28 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,polydipsia,0,9,R63.1,"[R63.1, F63.89, E23.2, F63.9, O40, G47.5, M79.89, R63.2, R06.1, H53.8, I44.2, G47.9, G47.8, G47.1, Q30.8, I45.9, R00.2, R06.81, G47.54, R06.8, M35.3]","[polydipsia [polydipsia], psychogenic polydipsia [other impulse disorders], primary polydipsia [diabetes insipidus], psychogenic polydipsia (disorder) [impulse disorder, unspecified], polyhydramnios [polyhydramnios], parasomnia [parasomnia], polyalgia [other specified soft tissue disorders], polyphagia [polyphagia], biphasic stridor [stridor], oscillopsia [other visual disturbances], parasystole (disorder) [atrioventricular block, complete], dyssomnia [sleep disorder, unspecified], parasomnia (disorder) [other sleep disorders], hypersomnia (disorder) [hypersomnia], polyrrhinia [other congenital malformations of nose], palpitations - irregular [conduction disorder, unspecified], intermittent palpitations [palpitations], vagal apnea [apnea, not elsewhere classified], secondary parasomnia [parasomnia in conditions classified elsewhere], vagal apnoea [other abnormalities of breathing], polymyalgia [polymyalgia rheumatica]]","[0.0000, 0.0455, 0.0635, 0.0786, 0.1474, 0.1520, 0.1553, 0.1573, 0.1641, 0.1675, 0.1648, 0.1600, 0.1662, 0.1667, 0.1727, 0.1702, 0.1687, 0.1672, 0.1730, 0.1700, 0.1729]","[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 23, 0, 0, 0, 0, 0, 0, 0, 96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40]"


## RxNorm pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

rxnorm_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp = nlp.LightPipeline(rxnorm_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[OK!]


In [ ]:
text = 'metformin 100 mg'

%time get_codes (rxnorm_lp, text, vocab='rxnorm_code')

CPU times: user 39.9 ms, sys: 14.5 ms, total: 54.3 ms
Wall time: 1.39 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,metformin 100 mg,0,15,861024,"[861024, 2200518, 141916, 334738, 332848, 104494, 861026, 199955, 333262, 429178, 439563, 103910, 429408, 450523, 1744000, 401938, 402346, 485246, 1726496, 198758, 316350, 213092, 858858, 245936, 336846]","[metformin hydrochloride 100 mg/ml [metformin hydrochloride 100 mg/ml], metformin hydrochloride 100 MG/ML Extended Release Suspension, fenofibrate 100 MG Oral Capsule, fenofibrate 100 mg [fenofibrate 100 mg], ciprofibrate 100 mg [ciprofibrate 100 mg], ciprofibrate 100 MG Oral Tablet, metformin hydrochloride 100 mg/ml [riomet] [metformin hydrochloride 100 mg/ml [riomet]], rutin 100 MG Oral Tablet, rutin 100 mg [rutin 100 mg], fendiline 100 MG Oral Tablet, fendiline 100 mg [fendiline 100 mg], carbamazepine 100 MG Oral Tablet [Epimaz], perazine 100 MG Oral Tablet, perazine 100 mg [perazine 100 mg], emtricitabine 100 mg [emtricitabine 100 mg], miglustat 100 MG Oral Capsule, miglustat 100 mg [miglustat 100 mg], azacitidine 100 MG Injection, azacitidine 100 mg [azacitidine 100 mg], niacin 100 MG Oral Capsule, niacin 100 mg [niacin 100 mg], dolasetron 100 MG Oral Tablet [Anzemet], carnosine 100 mg [carnosine 100 mg], trimebutine 100 MG Oral Tablet, trimebutine 100 mg [trimebutine 100 mg]]","[0.0658, 0.0658, 0.0684, 0.0684, 0.0697, 0.0697, 0.0770, 0.0765, 0.0765, 0.0775, 0.0775, 0.0780, 0.0804, 0.0804, 0.0818, 0.0811, 0.0811, 0.0812, 0.0812, 0.0811, 0.0811, 0.0836, 0.0828, 0.0844, 0.0844]"


In [ ]:
text = 'Coumadin 5 mg'

%time get_codes (rxnorm_lp, text, vocab='rxnorm_code')

CPU times: user 36.6 ms, sys: 11.8 ms, total: 48.4 ms
Wall time: 1.33 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Coumadin 5 mg,0,12,855333,"[855333, 432467, 438740, 153692, 352120, 1036890, 104363, 201269, 351399, 668467, 668694, 200795, 153618, 108781, 2475857, 2475858, 905173, 429252, 450946, 1091499, 246155, 439105, 1191685, 206956, 207641]","[warfarin sodium 5 MG [Coumadin], coumarin 5 MG Oral Tablet, coumarin 5 mg [coumarin 5 mg], oxybutynin chloride 5 MG Oral Tablet [Urimin], nitisinone 5 MG Oral Capsule [Orfadin], protriptyline hydrochloride 5 MG [Concordin], terazosin 5 MG Oral Tablet [Hytrin], metolazone 5 MG Oral Tablet [Metenix], prednisone 5 MG Oral Tablet [Prednicot], loratadine 5 MG [Claritin], bisacodyl 5 MG [Veracolate], bendroflumethiazide 5 MG Oral Tablet [Urizid], oxybutynin chloride 5 MG Oral Tablet [Contimin], nifedipine 5 MG Oral Capsule [Calanif], vericiguat 5 mg [vericiguat 5 mg], vericiguat 5 MG Oral Tablet, protriptyline hydrochloride 5 MG [Vivactil], pridinol 5 MG Oral Tablet, pridinol 5 mg [pridinol 5 mg], methylphenidate hydrochloride 5 MG [Ritalin], propinox 5 MG Oral Tablet, propinox 5 mg [propinox 5 mg], prednisolone 5 MG [Prednistab], prednisone 5 MG Oral Tablet [Prednicen-M], prochlorperazine 5 MG Rectal Suppository [Compazine]]","[0.0000, 0.0287, 0.0287, 0.0479, 0.0518, 0.0525, 0.0578, 0.0585, 0.0622, 0.0643, 0.0636, 0.0634, 0.0647, 0.0630, 0.0647, 0.0647, 0.0662, 0.0666, 0.0666, 0.0670, 0.0684, 0.0684, 0.0692, 0.0703, 0.0698]"


## NDC pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

ndc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_ndc", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("ndc_code")\
      .setDistanceFunction("EUCLIDEAN")

ndc_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        ndc_resolver])

ndc_lp = nlp.LightPipeline(ndc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_ndc download started this may take some time.
[OK!]


In [ ]:
text = 'metformin 500 mg'

%time get_codes (ndc_lp, text, vocab='ndc_code')

CPU times: user 32.6 ms, sys: 8.09 ms, total: 40.7 ms
Wall time: 887 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,metformin 500 mg,0,15,62207-0491,"[62207-0491, 50742-0633, 49483-0622, 62207-0416, 67296-0712, 70010-0063, 75834-0500, 61919-0769, 71800-0008, 70010-0491, 79739-7076, 70882-0124, 42571-0360, 42806-0405, 0069-0136, 49483-0382, 67253-0660, 34645-2105, 59630-0574]","[metformin er 500 mg, metformin 500 mg/1, metformin hcl 500 mg/1, metformin hcl 500 mg/kg, metformin hydrochloride 500 mg/1, metformin hydrochloride 500 mg/500mg, metformin hydrochloride er 500 mg/1, metformin hydrochloride e/r 500 mg/1, metformin hydrochloride 500 mg/5ml, metformin 500 mg/1 tablet, nabumetone 500 mg, metformin hcl 500 mg/1 tablet, metformin hydrochloride oral solution 500 mg/5ml, metformin hydrochloride tablets 500 mg/1, bosulif 500 mg/1, aspirin 500 mg/5001, pyrazinamide 500 mg/1, glycerin 500 mg/ml, fortamet 500 mg/1]","[0.0183, 0.0191, 0.0355, 0.0440, 0.0667, 0.0677, 0.0716, 0.0766, 0.0795, 0.1000, 0.1024, 0.1023, 0.1046, 0.1083, 0.1117, 0.1116, 0.1133, 0.1145, 0.1162]"


In [ ]:
text = 'aspirin 81 mg'

%time get_codes (ndc_lp, text, vocab='ndc_code')

CPU times: user 36 ms, sys: 6.7 ms, total: 42.7 ms
Wall time: 860 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,aspirin 81 mg,0,12,41250-0780,"[41250-0780, 72036-0080, 17714-0009, 49483-0349, 73089-0081, 46122-0598, 43063-0862, 69168-0318, 68210-0015, 49035-0467, 51824-0055, 68210-0150, 55910-0274, 67651-0361, 41520-0274, 36800-0467, 69168-0288, 52124-0012, 54257-0274, 70692-0126, 49638-0101, 55910-0467]","[aspirin 81 mg, aspirin 81mg, aspirin 81 mg/1, aspirin 81 mg/811, aspirin 81 mg/81mg, aspirin 81 81 mg/1, aspirin ec 81 mg/1, aspirin 81 mg tablet, assured aspirin 81 mg/1, equate aspirin 81 mg/1, careall aspirin 81 mg/1, valumeds aspirin 81 mg/1, rexall aspirin 81 mg/1, aspirin 81 mg/1 capsule, careone aspirin 81 mg/1, topcare aspirin 81 mg/1, aspirin chewable 81 mg/1, chewable aspirin 81 mg/1, enteric coated aspirin 81 mg/1, aspirin 81mg tablet, aspica (aspirin) 81 mg/1, dg health aspirin 81 mg/1]","[0.0000, 0.0152, 0.0181, 0.0253, 0.0317, 0.0323, 0.0338, 0.0353, 0.0389, 0.0388, 0.0411, 0.0416, 0.0422, 0.0503, 0.0546, 0.0547, 0.0574, 0.0631, 0.0644, 0.0643, 0.0672, 0.0677]"


## CPT pipeline

**Please note that you CAN NOT download this model using `.pretrained` method. To utilize this model, you need to have a valid CPT license. If you possess one and wish to use this model, kindly contact us at support@johnsnowlabs.com**

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

cpt_resolver = medical.SentenceEntityResolverModel.load("sbiobertresolve_cpt_procedures_measurements_augmented") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("cpt_code")\
      .setDistanceFunction("EUCLIDEAN")

cpt_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        cpt_resolver])

cpt_lp = nlp.LightPipeline(cpt_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_cpt_procedures_measurements_augmented download started this may take some time.
[OK!]


In [ ]:
text = 'heart surgery'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 15.3 ms, sys: 8.24 ms, total: 23.5 ms
Wall time: 317 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,heart surgery,0,12,33999,"[33999, 33258, 1018145, 0304T, 0167T, 93583, 1006057, 33736, 1013106, 33944, 33310, 33548, 33945, 62319, 33430]","[heart surgery [Unlisted procedure, cardiac surgery], Cardiac surgery procedure [Operative tissue ablation and reconstruction of atria, performed at the time of other cardiac procedure(s), extensive (eg, maze procedure), without cardiopulmonary bypass (List separately in addition to code for primary procedure)], Other Cardiac Surgery Procedures [Other Cardiac Surgery Procedures], Heart procedure [Insertion or removal and replacement of intracardiac ischemia monitoring system including imaging supervision and interpretation when performed and intra-operative interrogation and programming when performed; device only], Heart implantation [Transmyocardial transcatheter closure of ventricular septal defect, with implant; with cardiopulmonary bypass], Operation on cardiac septum [Percutaneous transcatheter septal reduction therapy (eg, alcohol septal ablation) including temporary pacemaker insertion when performed], Surgical Procedures on the Heart and Pericardium [Surgical Procedures on the Heart and Pericardium], Open heart atrial septostomy with cardiopulmonary bypass [Atrial septectomy or septostomy; open heart with cardiopulmonary bypass], Repair Procedures of Structural Heart Defect [Repair Procedures of Structural Heart Defect], Preparation of donor heart for transplantation [Backbench standard preparation of cadaver donor heart allograft prior to transplantation, including dissection of allograft from surrounding soft tissues to prepare aorta, superior vena cava, inferior vena cava, pulmonary artery, and left atrium for im...], Exploratory cardiotomy [Cardiotomy, exploratory (includes removal of foreign body, atrial or ventricular thrombus); without bypass], Surgical ventricular restoration procedure [Surgical ventricular restoration procedure, includes prosthetic patch, when performed (eg, ventricular remodeling, SVR, SAVER, Dor procedures)], Heart transplant [Heart transplant, with or without recipient cardiectomy], Insertion of catheter into heart chamber [Injection(s), including indwelling catheter placement, continuous infusion or intermittent bolus, of diagnostic or therapeutic substance(s) (including anesthetic, antispasmodic, opioid, steroid, other solution), not including neurolytic substances, includ], Replacement of mitral valve with cardiopulmonary bypass [Replacement, mitral valve, with cardiopulmonary bypass]]","[0.0000, 0.1031, 0.1078, 0.1377, 0.1574, 0.1645, 0.1703, 0.1780, 0.1779, 0.1784, 0.1891, 0.1871, 0.1890, 0.1904, 0.1932]"


In [ ]:
text = 'ct abdomen'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 10.1 ms, sys: 6.44 ms, total: 16.5 ms
Wall time: 306 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,ct abdomen,0,9,74150,"[74150, 35182, 70481, 62284, 74183, 64760, 76497, 61596, 70486, 76700, 31587, 35092, 0157T, 74175, 70490, 74160, 1002900, 71250, 49436]","[CT scan abdomen [Computed tomography, abdomen; without contrast material], RPR CONGENITAL AV FISTULA THORAX & ABDOMEN [Repair, congenital arteriovenous fistula; thorax and abdomen], CT of orbits [Computed tomography, orbit, sella, or posterior fossa or outer, middle, or inner ear; with contrast material(s)], CT of spine [Injection procedure for myelography and/or computed tomography, lumbar], MRI of abdomen [Magnetic resonance (eg, proton) imaging, abdomen; without contrast material(s), followed by with contrast material(s) and further sequences], Incision or removal of nerve to abdomen [Transection or avulsion of; vagus nerve (vagotomy), abdominal], CT scan [Unlisted computed tomography procedure (eg, diagnostic, interventional)], TRANSCOCHLR POST CRNL FOSSA W/WO MOBIL NRV/ART [Transcochlear approach to posterior cranial fossa, jugular foramen or midline skull base, including labyrinthectomy, decompression, with or without mobilization of facial nerve and/or petrous carotid artery], CT scan of face [Computed tomography, maxillofacial area; without contrast material], US abdominal scan [Ultrasound, abdominal, real time with image documentation; complete], Cricoid split procedure [Laryngoplasty, cricoid split, without graft placement], Procedure on artery of abdomen (procedure) [Direct repair of aneurysm, pseudoaneurysm, or excision (partial or total) and graft insertion, with or without patch graft; for ruptured aneurysm, abdominal aorta involving visceral vessels (mesenteric, celiac, renal)], Opening of abdomen [Laparotomy, implantation or replacement of gastric stimulation electrodes, lesser curvature (ie, morbid obesity)], CT of abdominal vascular structures (procedure) [Computed tomographic angiography, abdomen, with contrast material(s), including noncontrast images, if performed, and image postprocessing], CT scan of neck [Computed tomography, soft tissue neck; without contrast material], CT of abdomen with contrast [Computed tomography, abdomen; with contrast material(s)], Anesthesia for Procedures on the Upper Abdomen [Anesthesia for Procedures on the Upper Abdomen], Diagnostic CT scan of chest [Computed tomography, thorax, diagnostic; without contrast material], Creation of exit site for catheter in abdominal cavity [Delayed creation of exit site from embedded subcutaneous segment of intraperitoneal cannula or catheter]]","[0.0756, 0.0985, 0.1099, 0.1268, 0.1351, 0.1422, 0.1526, 0.1571, 0.1570, 0.1534, 0.1619, 0.1549, 0.1559, 0.1614, 0.1638, 0.1645, 0.1591, 0.1694, 0.1672]"


In [ ]:
text = 'Left heart cath'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 12.4 ms, sys: 7.71 ms, total: 20.2 ms
Wall time: 318 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Left heart cath,0,14,93462,"[93462, 33415, 93607, 33988, 33989, 93620, 93514, 36014, 93452, 0281T, 33975, 93531]","[Cardiac catheterization, left heart [Left heart catheterization by transseptal puncture through intact septum or by transapical puncture (List separately in addition to code for primary procedure)], Incision or removal of tissue below left lower heart valve [Resection or incision of subvalvular tissue for discrete subvalvular aortic stenosis], Left ventricular recording (Deprecated) [Left ventricular recording (Deprecated)], Insertion of left heart vent through chest for blood oxygenation rewarming and return [Insertion of left heart vent by thoracic incision (eg, sternotomy, thoracotomy) for ECMO/ECLS], RMVL LEFT HEART VENT BY THORACIC INCIS ECMO/ECLS [Removal of left heart vent by thoracic incision (eg, sternotomy, thoracotomy) for ECMO/ECLS], Left ventricular recording [Comprehensive electrophysiologic evaluation including insertion and repositioning of multiple electrode catheters with induction or attempted induction of arrhythmia; with right atrial pacing and recording, right ventricular pacing and recording, His bund], Left heart catheterization by left ventricular puncture (Deprecated) [Left heart catheterization by left ventricular puncture (Deprecated)], Insertion of catheter into left pulmonary artery [Selective catheter placement, left or right pulmonary artery], Insertion of catheter into left heart including imaging interpretation and supervision and injection [Left heart catheterization including intraprocedural injection(s) for left ventriculography, imaging supervision and interpretation, when performed], Plication of left atrial appendage [Percutaneous transcatheter closure of the left atrial appendage with implant, including fluoroscopy, transseptal puncture, catheter placement(s), left atrial angiography, left atrial appendage angiography, radiological supervision and interpretation], Insertion of external lower heart chamber blood flow assist device [Insertion of ventricular assist device; extracorporeal, single ventricle], Cardiac catheterization, combined right and left heart [Combined right heart catheterization and retrograde left heart catheterization, for congenital cardiac anomalies]]","[0.1060, 0.1104, 0.1272, 0.1367, 0.1424, 0.1433, 0.1611, 0.1639, 0.1683, 0.1678, 0.1731, 0.1687]"


## SNOMED pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

snomed_ct_resolver = medical.SentenceEntityResolverModel\
      .pretrained("sbiobertresolve_snomed_findings_aux_concepts","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

snomed_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        snomed_ct_resolver])

snomed_lp = nlp.LightPipeline(snomed_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_snomed_findings_aux_concepts download started this may take some time.
[OK!]


In [ ]:
text = 'bladder cancer'

%time get_codes (snomed_lp, text, vocab='snomed_code', aux_label=True)

CPU times: user 28.5 ms, sys: 13.4 ms, total: 41.9 ms
Wall time: 1.43 s


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,bladder cancer,0,13,1259400001,"[1259400001, 315269000, 1259401002, 1237457005, 1263639008, 27090000, 1237465008, 1259402009, 1208460000, 53530009, 1259705005, 12400006, 315271000, 309273004]","[primary carcinoma of bladder, suspected bladder cancer, primary transitional cell carcinoma of bladder, secondary carcinoma of bladder, secondary transitional cell carcinoma of bladder, urothelial carcinoma, secondary adenocarcinoma of bladder, primary adenocarcinoma of bladder, sarcomatoid urothelial carcinoma of urinary bladder, urothelial carcinoma in situ, primary sarcomatoid urothelial carcinoma of urinary bladder, papillary urothelial carcinoma, suspected urothelial cancer, bladder biopsy specimen]","[No_Concept_Class, Context-dependent, No_Concept_Class, No_Concept_Class, No_Concept_Class, Morph Abnormality, No_Concept_Class, No_Concept_Class, No_Concept_Class, Morph Abnormality, No_Concept_Class, Morph Abnormality, Context-dependent, Specimen]","[0.0790, 0.0904, 0.0985, 0.1035, 0.1039, 0.1091, 0.1118, 0.1218, 0.1292, 0.1429, 0.1442, 0.1513, 0.1547, 0.1527]"


In [ ]:
text = 'schizophrenia'

%time get_codes (snomed_lp, text, vocab='snomed_code', aux_label=True)

CPU times: user 29.4 ms, sys: 12.5 ms, total: 41.9 ms
Wall time: 1.44 s


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,schizophrenia,0,12,160328009,"[160328009, 161468000, 702919008, 64984005, 1204417003, 106601008, 47862007, 266969002, 394816006, 1259128002, 446949003, 160324006, 446167009, 113236004, 10784006, 11230001000004106]","[fh: schizophrenia, h/o: schizophrenia, schizophrenia clinic, schizozygia, early onset schizophrenia, schizopyrenida, schizozygine, relative - psychotic illness, mental illness (specialty), alzheimer disease with psychosis, psychotic symptom rating scales total score, fh - mental disorder, psychotic symptom rating scale, symphysis mentalis, antipsychotic drug, history of psychosis]","[Context-dependent, Context-dependent, Location, Organism, No_Concept_Class, Organism, Substance, Context-dependent, Qualifier Value, No_Concept_Class, Observable Entity, Context-dependent, Staging / Scales, Body Structure, Pharma/Biol Product, Context-dependent]","[0.0597, 0.0653, 0.0974, 0.1137, 0.1199, 0.1386, 0.1579, 0.1524, 0.1678, 0.1746, 0.1817, 0.1767, 0.1863, 0.1924, 0.1895, 0.1797]"


## LOINC Pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

loinc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_loinc_augmented", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("loinc_code")\
      .setDistanceFunction("EUCLIDEAN")

loinc_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        loinc_resolver])

loinc_lp = nlp.LightPipeline(loinc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_loinc_augmented download started this may take some time.
[OK!]


In [ ]:
text = 'FLT3 gene mutation analysis'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 48.3 ms, sys: 19.6 ms, total: 67.9 ms
Wall time: 1 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,FLT3 gene mutation analysis,0,26,47958-4,"[47958-4, LP310331-6, 21676-2, 53863-7, 92843-2, 57963-1, 96968-3, 49873-3, 58987-9, 53840-5, 99129-9, 81816-1, 72520-0, 22074-9, 21740-6, 34499-4, 49704-0, 46225-9, 38532-8, 71358-6, 81831-0, 53895-9, 35298-9, 49872-5, 48972-4]","[FLT3 gene targeted mutation analysis [FLT3 gene targeted mutation analysis], FLT3 gene p.Asp835 mutations [FLT3 gene p.Asp835 mutations], FGFR3 gene targeted mutation analysis [FGFR3 gene targeted mutation analysis], FLNA gene targeted mutation analysis [FLNA gene targeted mutation analysis], FLT3 gene.p.Asp835 mutations [FLT3 gene.p.Asp835 mutations], FLCN gene targeted mutation analysis [FLCN gene targeted mutation analysis], TNFAIP3 gene targeted mutation analysis [TNFAIP3 gene targeted mutation analysis], POU3F4 gene targeted mutation analysis [POU3F4 gene targeted mutation analysis], ITGB3 gene targeted mutation analysis [ITGB3 gene targeted mutation analysis], TGFB3 gene targeted mutation analysis [TGFB3 gene targeted mutation analysis], SF3B1 gene targeted mutation analysis [SF3B1 gene targeted mutation analysis], OPA3 gene targeted mutation analysis [OPA3 gene targeted mutation analysis], FLT3 gene.p.Asp835+Ile836 mutations [FLT3 gene.p.Asp835+Ile836 mutations], FGFR3 gene mutations tested for [FGFR3 gene mutations tested for], TRAF3 gene targeted mutation analysis [TRAF3 gene targeted mutation analysis], GPC3 gene targeted mutation analysis [GPC3 gene targeted mutation analysis], SH3TC2 gene targeted mutation analysis [SH3TC2 gene targeted mutation analysis], ABCA3 gene targeted mutation analysis [ABCA3 gene targeted mutation analysis], SPG3A gene targeted mutation analysis [SPG3A gene targeted mutation analysis], GJB3 gene targeted mutation analysis [GJB3 gene targeted mutation analysis], LAMB3 gene targeted mutation analysis [LAMB3 gene targeted mutation analysis], SH3BP2 gene targeted mutation analysis [SH3BP2 gene targeted mutation analysis], FXN gene targeted mutation analysis [FXN gene targeted mutation analysis], FH gene targeted mutation analysis [FH gene targeted mutation analysis], FGFR2 gene+FGFR3 gene targeted mutation analysis [FGFR2 gene+FGFR3 gene targeted mutation analysis]]","[0.0354, 0.1032, 0.1021, 0.1111, 0.1155, 0.1135, 0.1182, 0.1258, 0.1249, 0.1285, 0.1303, 0.1255, 0.1308, 0.1316, 0.1287, 0.1328, 0.1346, 0.1315, 0.1351, 0.1352, 0.1339, 0.1390, 0.1347, 0.1318, 0.1387]"


In [ ]:
text = 'Hematocrit'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 237 ms, sys: 21.7 ms, total: 258 ms
Wall time: 1.1 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Hematocrit,0,9,11151-8,"[11151-8, 16931-8, 32354-3, 20570-8, 11153-4, 39227-4, 42908-4, 30899-9, 71832-0, 4593-0, 14775-1, 41654-5, LP35956-9, 18277-4, 48703-3, 71828-8, 8478-0, 41655-2, 53224-2, 41986-1, LP30932-5, 76283-1, 34663-5, 10346-5, LP14154-6]","[Hematocrit [Hematocrit], Hematocrit/Hemoglobin [Hematocrit/Hemoglobin], Hematocrit [Volume Fraction] of Arterial blood [Hematocrit], Hematocrit [Volume Fraction] of Blood [Hematocrit], Hematocrit [Volume Fraction] of Body fluid [Hematocrit], Hematocrit test status [Hematocrit test status], Hematocrit [Volume Fraction] of Capillary blood [Hematocrit], Blood volume [Blood volume], Hematocrit [Pure volume fraction] of Arterial blood [Hematocrit], Hemoglobin I [Hemoglobin I], Hemoglobin [Hemoglobin], Hematocrit [Volume Fraction] of Venous blood [Hematocrit], Hematocrit test status/results [Hematocrit test status/results], Hemoglobin F [Hemoglobin F], Hematocrit [Volume Fraction] of Blood by Estimated [Hematocrit], Hematocrit [Pure volume fraction] of Body fluid [Hematocrit], Mean blood pressure [Intravascular mean], Hematocrit [Volume Fraction] of Mixed venous blood [Hematocrit], Hemoglobin M [Hemoglobin M], Hematocrit test status &or results [Hematocrit test status &or results], Hemoglobin N [Hemoglobin N], Blood flow rate [Blood flow rate], Hemoglobin S [Hemoglobin S], Hemoglobin A [Hemoglobin A], Hemoglobin and Hematocrit panel [Hemoglobin and Hematocrit panel]]","[0.0000, 0.0506, 0.0590, 0.0625, 0.0675, 0.0740, 0.1035, 0.1010, 0.1075, 0.1082, 0.1122, 0.1139, 0.1162, 0.1166, 0.1196, 0.1194, 0.1193, 0.1245, 0.1245, 0.1267, 0.1257, 0.1229, 0.1279, 0.1304, 0.1324]"


In [ ]:
text = 'vomiting'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 36.9 ms, sys: 13.9 ms, total: 50.8 ms
Wall time: 871 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,vomiting,0,7,28397-8,"[28397-8, LP7717-4, 81224-8, 28391-1, LA15902-2, 94070-0, LA25131-6, 81660-3, LA17127-4, 94069-2, 42848-2, LA33070-6, LA6150-2, LA30521-1, LA16987-2, LA30520-3, LA29193-2, 79553-4, 100783-0, LP74881-1, LA7428-1, LA7449-7, LA27492-0, LA20938-9]","[Vomiting [Vomiting], Vomitus [Vomitus], Vomiting [CCC] [Vomiting], Nausea [Nausea], Nausea or vomiting [Nausea or vomiting], Emesis count [Emesis count], Rumbling [Rumbling], Nausea [Presence] [Nausea], Choking [Choking], Emesis occurred [Emesis occurred], Nausea [CCC] [Nausea], Oozing [Oozing], Agitation [Agitation], Groaning respiration [Groaning respiration], Fainting [Fainting], Grimaces [Grimaces], Crying [Crying], Fluid output emesis [#] [Fluid output.emesis], Crying spells [Crying spells], Gas /bloating [Gas /bloating], Dizziness [Dizziness], Irritation [Irritation], Cramping [Cramping], Emaciated [Emaciated]]","[0.0000, 0.0175, 0.0589, 0.0755, 0.0846, 0.0949, 0.0985, 0.0980, 0.1009, 0.1012, 0.1034, 0.1090, 0.1095, 0.1108, 0.1105, 0.1193, 0.1187, 0.1214, 0.1215, 0.1225, 0.1230, 0.1242, 0.1302, 0.1279]"


## HCPCS Pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

hcpcs_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_hcpcs", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("hcpcs_code")\
      .setDistanceFunction("EUCLIDEAN")

hcpcs_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hcpcs_resolver])

hcpcs_lp = nlp.LightPipeline(hcpcs_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_hcpcs download started this may take some time.
[OK!]


In [ ]:
text = "Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type"

%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 28.7 ms, sys: 9.73 ms, total: 38.5 ms
Wall time: 276 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,"Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type",0,104,L8001,"[L8001, L8002, L8000, L8033, L8032, S2067, L8020, S8080, L8010, G9131, G9071, S2066, G9075, K1005, G9073, L8031, G9072, L8015, E0602, L8600, S2068, E0604, G9074, L8030, G9114]","[Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type, Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, bilateral, any size, any type, Breast prosthesis, mastectomy bra, without integrated breast prosthesis form, any size, any type, Nipple prosthesis, custom fabricated, reusable, any material, any type, each, Nipple prosthesis, prefabricated, reusable, any type, each, Breast reconstruction of a single breast with ""stacked"" deep inferior epigastric perforator (diep) flap(s) and/or gluteal artery perforator (gap) flap(s), including harvesting of the flap(s), microvascular transfer, closure of donor site(s) and shaping the flap into a breast, unilateral, Breast prosthesis, mastectomy form, Scintimammography (radioimmunoscintigraphy of the breast), unilateral, including supply of radiopharmaceutical, Breast prosthesis, mastectomy sleeve, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; extent of disease unknown, staging in progress, or not listed (for use in a medicare-approved demonstration project), Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage i or stage iia-iib; or t3, n1, m0; and er and/or pr positive; with no evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonstration project), Breast reconstruction with gluteal artery perforator (gap) flap, including harvesting of the flap, microvascular transfer, closure of donor site and shaping the flap into a breast, unilateral, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; m1 at diagnosis, metastatic, locally recurrent, or progressive (for use in a medicare-approved demonstration project), Disposable collection and storage bag for breast milk, any size, any type, each, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage iiia-iiib; and not t3, n1, m0; and er and/or pr positive; with no evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonstration project), Breast prosthesis, silicone or equal, with integral adhesive, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage i, or stage iia-iib; or t3, n1, m0; and er and pr negative; with no evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonstration project), External breast prosthesis garment, with mastectomy form, post mastectomy, Breast pump, manual, any type, Implantable breast prosthesis, silicone or equal, Breast reconstruction with deep inferior epigastric perforator (diep) flap or superficial inferior epigastric artery (siea) flap, including harvesting of the flap, microvascular transfer, closure of donor site and shaping the flap into a breast, unilateral, Breast pump, hospital grade, electric (ac and / or dc), any type, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage iiia-iiib; and not t3, n1, m0; and er and pr negative; with no evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonstration project), Breast prosthesis, silicone or equal, without integral ad

In [ ]:
text= "Dietary consultation for carbohydrate counting for type I diabetes."
%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 23.3 ms, sys: 11.3 ms, total: 34.5 ms
Wall time: 218 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Dietary consultation for carbohydrate counting for type I diabetes.,0,66,S9465,"[S9465, S9434, S9460, E0787, G8777, S9435, B4157, B5200, B4162, G8725, G8485, B4149, G0343, A4236, S9470, B5100, A4234, B4155, A4235, G0246, G9147, G4017, B5000, VM, G0342]","[Diabetic management program, dietitian visit, Modified solid food supplements for inborn errors of metabolism, Diabetic management program, nurse visit, External ambulatory infusion pump, insulin, dosage rate adjustment using therapeutic continuous glucose sensing, Diabetes screening test performed, Medical foods for inborn errors of metabolism, Enteral formula, nutritionally complete, for special metabolic needs for inherited disease of metabolism, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, stress-branch chain amino acids-freamine-hbc-premix, Enteral formula, for pediatrics, special metabolic needs for inherited disease of metabolism, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Fasting lipid profile performed (triglycerides, ldl-c, hdl-c and total cholesterol), I intend to report the diabetes mellitus (dm) measures group, Enteral formula, manufactured blenderized natural foods with intact nutrients, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Laparotomy for islet cell transplant, includes portal vein catheterization and infusion, Replacement battery, silver oxide, for use with medically necessary home blood glucose monitor owned by patient, each, Nutritional counseling, dietitian visit, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, hepatic, hepatamine-premix, Replacement battery, alkaline, j cell, for use with medically necessary home blood glucose monitor owned by patient, each, Enteral formula, nutritionally incomplete/modular nutrients, includes specific nutrients, carbohydrates (e.g., glucose polymers), proteins/amino acids (e.g., glutamine, arginine), fat (e.g., medium chain triglycerides) or combination, administered through an enteral feeding tube, 100 calories = 1 unit, Replacement battery, lithium, for use with medically necessary home blood glucose monitor owned by patient, each, Follow-up physician evaluation and management of a diabetic patient with diabetic sensory neuropathy resulting in a loss of protective sensation (lops) to include at least the following: (1) a patient history, (2) a physical examination that includes: (a) visual inspection of the forefoot, hindfoot and toe web spaces, (b) evaluation of protective sensation, (c) evaluation of foot structure and biomechanics, (d) evaluation of vascular status and skin integrity, and (e) evaluation and recommendation of footwear, and (3) patient education, Outpatient intravenous insulin treatment (oivit) either pulsatile or continuous, by any means, guided by the results of measurements for: respiratory quotient; and/or, urine urea nitrogen (uun); and/or, arterial, venous or capillary glucose; and/or potassium concentration, Nutrition/dietician mips specialty set, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, renal-aminosyn-rf, nephramine, renamine-premix, Medicare diabetes prevention program (mdpp) virtual make-up session, Laparoscopy for islet cell transplant, includes portal vein catheterization and infusion]","[0.2426, 0.3054, 0.3147, 0.3332, 0.3201, 0.3398, 0.

In [ ]:
text= "Psychiatric consultation for alcohol withdrawal and dependance."
%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 34.3 ms, sys: 2.04 ms, total: 36.3 ms
Wall time: 219 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Psychiatric consultation for alcohol withdrawal and dependance.,0,62,H0010,"[H0010, H0011, H0007, H0012, H0013, H0008, T1006, G9621, H0009, G2196, S9485, H2036, G2203, H0005, G8959, H2034, G2200, HG, H0014, H0006, G2121, H2018, T1007, G9717, H0022]","[Alcohol and/or drug services; sub-acute detoxification (residential addiction program inpatient), Alcohol and/or drug services; acute detoxification (residential addiction program inpatient), Alcohol and/or drug services; crisis intervention (outpatient), Alcohol and/or drug services; sub-acute detoxification (residential addiction program outpatient), Alcohol and/or drug services; acute detoxification (residential addiction program outpatient), Alcohol and/or drug services; sub-acute detoxification (hospital inpatient), Alcohol and/or substance abuse services, family/couple counseling, Patient identified as an unhealthy alcohol user when screened for unhealthy alcohol use using a systematic screening method and received brief counseling, Alcohol and/or drug services; acute detoxification (hospital inpatient), Patient identified as an unhealthy alcohol user when screened for unhealthy alcohol use using a systematic screening method, Crisis intervention mental health services, per diem, Alcohol and/or other drug treatment program, per diem, Documentation of medical reason(s) for not providing brief counseling if identified as an unhealthy alcohol user (e.g., limited life expectancy, other medical reasons), Alcohol and/or drug services; group counseling by a clinician, Clinician treating major depressive disorder communicates to clinician treating comorbid condition, Alcohol and/or drug abuse halfway house services, per diem, Patient identified as an unhealthy alcohol user received brief counseling, Opioid addiction treatment program, Alcohol and/or drug services; ambulatory detoxification, Alcohol and/or drug services; case management, Depression, anxiety, apathy, and psychosis assessed, Psychosocial rehabilitation services, per diem, Alcohol and/or substance abuse services, treatment plan development and/or modification, Documentation stating the patient has had a diagnosis of depression or has had a diagnosis of bipolar disorder, Alcohol and/or drug intervention service (planned facilitation)]","[0.1938, 0.2086, 0.2104, 0.2134, 0.2177, 0.2423, 0.2430, 0.2509, 0.2554, 0.2594, 0.2716, 0.2658, 0.2729, 0.2695, 0.2716, 0.2755, 0.2708, 0.2776, 0.2933, 0.2799, 0.2904, 0.3020, 0.2915, 0.2835, 0.2985]"


## UMLS Pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

umls_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_umls_major_concepts", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

umls_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        umls_resolver])

umls_lp = nlp.LightPipeline(umls_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_umls_major_concepts download started this may take some time.
[OK!]


In [ ]:
# medical device
text = 'X-Ray'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 32.1 ms, sys: 9.99 ms, total: 42.1 ms
Wall time: 725 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,X-Ray,0,4,C0043303,"[C0043303, C0184486, C0182605, C0183263, C0183264, C3880096, C0868115, C0221564, C0000766, C0184480, C3880545]","[x-ray film, x-ray tubes, x-ray machine, x-ray shield, x-ray aprons, x-ray system tube (physical object), exposure x ray, specific x-ray findings, x-ray abnormal, x-ray system generator (physical object), x-ray system cover (physical object)]","[0.0416, 0.0662, 0.0712, 0.0906, 0.1029, 0.1180, 0.1201, 0.1210, 0.1165, 0.1286, 0.1314]"


In [ ]:
# Injuries & poisoning
text = 'out-of-date food poisoning'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 24.3 ms, sys: 10.8 ms, total: 35.1 ms
Wall time: 686 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,out-of-date food poisoning,0,25,C0016479,"[C0016479, C0349783, C0411266, C0161721, C0417672, C0567797, C0411267, C1281398, C0178496, C4743928, C0343360, C3665356, C4724202, C1820738, C0565937, C4743931]","[food poisoning (disorder), infective food poisoning (disorder), chemical food poisoning (disorder), toxic effect of noxious substance eaten as food (disorder), accident due to lack of food, pectin poisoning of undetermined intent (disorder), heavy metal food poisoning (disorder), inanition due to lack of food (disorder), bacterial food poisoning (disorder), grade 2 feeding intolerance, viral food poisoning (disorder), history of food intolerance, feeding intolerance, feeding intolerance, food burn of skin (disorder), grade 5 feeding intolerance, adverse event]","[0.1007, 0.1104, 0.1117, 0.1331, 0.1431, 0.1496, 0.1519, 0.1546, 0.1548, 0.1585, 0.1625, 0.1618, 0.1624, 0.1624, 0.1645, 0.1660]"


In [ ]:
# clinical findings
text = 'type two diabetes mellitus'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 33.9 ms, sys: 6.32 ms, total: 40.3 ms
Wall time: 696 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,type two diabetes mellitus,0,25,C4014362,"[C4014362, C3532488, C1320657, C1313937, C0455488, C2733146, C4017629, C3532489, C3275844, C0241863, C1261139, C0421248, C5195213, C1317301, C3278636, C2675471, C0235397, C4316344, C3872445]","[type 2 diabetes mellitus (t2d), history of diabetes mellitus type 2 (situation), diabete type, fh: diabetes mellitus, pre-existing diabetes mellitus, type 2 diabetes mellitus uncontrolled, diabetes mellitus, type ii, digenic, history of diabetes mellitus type i, diabetes mellitus, type ii, susceptibility to, diabetic, diabetic relative, insulin diabetic, increased risk of type 2 diabetes, diabetes status, neonatal insulin-dependent diabetes mellitus, microvascular complications of diabetes, susceptibility to, 2, diabetes mellitus precipitated, disorder of vision due to secondary diabetes mellitus (disorder), suspected glaucoma due to type 2 diabetes mellitus (situation)]","[0.0438, 0.0568, 0.0618, 0.0697, 0.0780, 0.0891, 0.0910, 0.0883, 0.0940, 0.0939, 0.1031, 0.1062, 0.1087, 0.1104, 0.1116, 0.1201, 0.1138, 0.1198, 0.1243]"


## MeSH pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

mesh_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_mesh","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("mesh_code")\
      .setDistanceFunction("EUCLIDEAN")

mesh_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        mesh_resolver])

mesh_lp = nlp.LightPipeline(mesh_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_mesh download started this may take some time.
[OK!]


In [ ]:
text = 'chest pain '

%time get_codes (mesh_lp, text, vocab='mesh_code')

CPU times: user 68.6 ms, sys: 18.7 ms, total: 87.3 ms
Wall time: 1.26 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,chest pain,0,10,D002637,"[D002637, D059350, D019547, D020069, D015746, D000072716, D005157, D059265, D001416, D048949, D021501, D058447, D010146, D059390, C563475, D059352, D053591, D059787, D009209, D010148, D058748, D020918, D017699, D059388, D005409]","[Chest Pain, Chronic Pain, Neck Pain, Shoulder Pain, Abdominal Pain, Cancer Pain, Facial Pain, Visceral Pain, Back Pain, Labor Pain, Flank Pain, Eye Pain, Pain, Breakthrough Pain, Paroxysmal Extreme Pain Disorder, Musculoskeletal Pain, Pain, Referred, Acute Pain, Myofascial Pain Syndromes, Pain, Intractable, Pain Perception, Complex Regional Pain Syndromes, Pelvic Pain, Pelvic Girdle Pain, Flail Chest]","[0.0000, 0.0577, 0.0587, 0.0601, 0.0658, 0.0704, 0.0712, 0.0741, 0.0766, 0.0778, 0.0794, 0.0827, 0.0845, 0.0914, 0.0941, 0.0935, 0.0957, 0.1023, 0.1078, 0.1060, 0.1109, 0.1166, 0.1187, 0.1189, 0.1199]"


In [ ]:
text = 'pericardectomy'

%time get_codes (mesh_lp, text, vocab='mesh_code')

CPU times: user 60.6 ms, sys: 21.2 ms, total: 81.8 ms
Wall time: 1.23 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,pericardectomy,0,13,D010492,"[D010492, D011670, D018700, D014068, D000541, D005927, D020884, D064727, D010611, D011672, D059526, D001062, D002431, D011013, D013156, D032801, D001047, D020519, D000074885, D038421, D010180, D007825, D000074406, D017128, D056346]","[Pericardiectomy, Pulpectomy, Pleurodesis, Tonsillectomy, Alveolectomy, Glossectomy, Colpotomy, Posterior Capsulotomy, Pharyngectomy, Pulpotomy, Foraminotomy, Appendectomy, Cecostomy, Pneumonectomy, Splenectomy, Iridectomy, Apicoectomy, Pericardiocentesis, Amniotomy, Hemispherectomy, Pancreatectomy, Laryngectomy, Mastoidectomy, Embolectomy, Sternotomy]","[0.0732, 0.1287, 0.1499, 0.1540, 0.1573, 0.1582, 0.1645, 0.1622, 0.1660, 0.1676, 0.1729, 0.1697, 0.1744, 0.1774, 0.1753, 0.1757, 0.1783, 0.1812, 0.1803, 0.1726, 0.1799, 0.1813, 0.1852, 0.1909, 0.1867]"


## HPO Pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

hpo_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_HPO", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

hpo_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hpo_resolver])

hpo_lp = nlp.LightPipeline(hpo_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_HPO download started this may take some time.
[OK!]


In [ ]:
text = 'bladder cancer'

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 62.2 ms, sys: 19.8 ms, total: 82.1 ms
Wall time: 528 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,HP:0009725,"[HP:0009725, HP:0002862, HP:0006740, HP:0030409, HP:0100517, HP:0000015, HP:0100516, HP:0009726, HP:0004375, HP:0004321, HP:0032465, HP:0010786, HP:0010474, HP:0010784, HP:0006762, HP:0005584, HP:0025489, HP:0010477]","[bladder cancer, bladder carcinoma, transitional cell carcinoma of the bladder, renal urothelial carcinoma, neoplasia of the urethra, bladder diverticulum, ureter, cancer of, kidney cancer, nervous system cancer, bladder fistula, bladder trabeculation, urinary tract neoplasia, bladder stones, uterine cancer, renal pelvic carcinoma, renal carcinoma, bladder duplication, aplasia of the bladder]","[0.0000, 0.0503, 0.0905, 0.1612, 0.1729, 0.1738, 0.1839, 0.1886, 0.1924, 0.1892, 0.1930, 0.1903, 0.1904, 0.1938, 0.1987, 0.2013, 0.2006, 0.2039]"


In [ ]:
text = 'bipolar disorder'

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 67 ms, sys: 25 ms, total: 92 ms
Wall time: 495 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bipolar disorder,0,15,HP:0007302,"[HP:0007302, HP:0100753, HP:0012076, HP:0000708, HP:0033516, HP:0012077, HP:0000709, HP:0012075, HP:0000716, HP:0000722, HP:0002134, HP:0011511, HP:0000725, HP:0025102, HP:0002072, HP:0011223, HP:0033512, HP:0100022, HP:0001345, HP:0002269]","[bipolar disorder, schizophrenia, borderline personality disorder, psychiatric disorders, benzodiazepine dependence, histrionic personality disorder, psychosis, personality disorder, depressive disorder, obsessive compulsive disorder, basal ganglia disease, macular schisis, psychotic episodes, basal ganglia dysgenesis, choreatic disease, metopic depression, stimulant dependence, movement disorder, psychotic mentation, migrational brain disorder]","[0.0000, 0.1488, 0.1486, 0.1577, 0.1740, 0.1731, 0.1784, 0.1764, 0.1786, 0.1830, 0.1925, 0.2145, 0.2192, 0.2182, 0.2376, 0.2325, 0.2325, 0.2330, 0.2443, 0.2393]"


In [ ]:
text = 'schizophrenia '

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 55.6 ms, sys: 20.9 ms, total: 76.5 ms
Wall time: 460 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,schizophrenia,0,13,HP:0100753,"[HP:0100753, HP:0000709, HP:0007302, HP:0000708, HP:0000725, HP:0011511, HP:0012077, HP:0001345, HP:0030325, HP:0012075, HP:0000722, HP:0033512, HP:0012076, HP:0100923, HP:0002072, HP:0033715, HP:0010891, HP:0011626, HP:0100656, HP:0033516, HP:0008404, HP:0002269]","[schizophrenia, psychosis, bipolar disorder, psychiatric disorders, psychotic episodes, macular schisis, histrionic personality disorder, psychotic mentation, cervicomedullary schisis, personality disorder, obsessive compulsive disorder, stimulant dependence, borderline personality disorder, clavicular sclerosis, choreatic disease, mesial temporal sclerosis, scheuermann disease, scimitar syndrome, thoracoabdominal schisis, benzodiazepine dependence, onychodystrophy, migrational brain disorder]","[0.0000, 0.1172, 0.1488, 0.1599, 0.1877, 0.1898, 0.1923, 0.2033, 0.2163, 0.2113, 0.2166, 0.2257, 0.2202, 0.2345, 0.2431, 0.2389, 0.2497, 0.2438, 0.2578, 0.2483, 0.2604, 0.2455]"


## All the resolvers in the same pipeline
### (just to show how it is done..  will not be used in this notebook)

In [ ]:

documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

snomed_ct_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

icd10_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

cpt_resolver = medical.SentenceEntityResolverModel.load("sbiobertresolve_cpt_procedures_augmented") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("cpt_code")\
      .setDistanceFunction("EUCLIDEAN")

hcc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_hcc_code")\
      .setDistanceFunction("EUCLIDEAN")

loinc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_loinc_augmented", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("loinc_code")\
      .setDistanceFunction("EUCLIDEAN")

hcpcs_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_hcpcs", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("hcpcs_code")\
      .setDistanceFunction("EUCLIDEAN")

umls_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_umls_major_concepts", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

hpo_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_HPO", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

mesh_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_mesh","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("mesh_code")\
      .setDistanceFunction("EUCLIDEAN")

ndc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_ndc", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("ndc_code")\
      .setDistanceFunction("EUCLIDEAN")


resolver_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        icd10_resolver,
        hcc_resolver,
        rxnorm_resolver,
        ndc_resolver,
        cpt_resolver,
        snomed_ct_resolver,
        loinc_resolver,
        hcpcs_resolver,
        umls_resolver,
        mesh_resolver,
        hpo_resolver])

resolver_lp = nlp.LightPipeline(resolver_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_snomed_findings download started this may take some time.
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
[OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[OK!]
sbiobertresolve_cpt_procedures_augmented download started this may take some time.
[OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[OK!]
sbiobertresolve_loinc_augmented download started this may take some time.
[OK!]
sbiobertresolve_hcpcs download started this may take some time.
[OK!]
sbiobertresolve_umls_major_concepts download started this may take some time.
[OK!]
sbiobertresolve_HPO download started this may take some time.
[OK!]
sbiobertresolve_mesh download started this may take some time.
[OK!]
sbiobertresolve_ndc download started this may take some time.
[OK!]


## Getting some predictions from resolvers

In [ ]:
icd_chunks = ['obesity',
'a respiratory tract infection',
'basal cell carcinoma of skin',
'acute maxillary sinusitis',
'chronic kidney disease stage',
'diabetes mellitus type 2',
'lymph nodes of multiple sites',
'other chronic pain',
'severe abdominal pain',
'squamous cell carcinoma of skin',
'type 2 diabetes mellitus']

snomed_chunks= ['down syndrome', 'adenocarcinoma', 'aortic valve stenosis',
       'atherosclerosis', 'atrial fibrillation',
       'hypertension', 'lung cancer', 'seizure',
       'squamous cell carcinoma', 'stage IIIB', 'mediastinal lymph nodes']

In [ ]:
from IPython.display import display

for chunk in icd_chunks:

    print ('>> ',chunk)
    display(get_codes (icd_lp, chunk, vocab='icd10cm_code'))

>>  obesity


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,obesity,0,6,E66.9,"[E66.9, E66.8, Z68.41, Q13.0, E66, E66.01, Z86.39, E34.9, H35.50, Z83.49, Q55.62, E66.3]","[obesity [obesity, unspecified], abdominal obesity [other obesity], obese [body mass index [bmi] 40.0-44.9, adult], central obesity [coloboma of iris], overweight and obesity [overweight and obesity], morbid obesity [morbid (severe) obesity due to excess calories], h/o: obesity [personal history of other endocrine, nutritional and metabolic disease], severe obesity [endocrine disorder, unspecified], centripetal obesity [unspecified hereditary retinal dystrophy], fh: obesity [family history of other endocrine, nutritional and metabolic diseases], truncal obesity [hypoplasia of penis], overweight [overweight]]","[0.0000, 0.0218, 0.0264, 0.0278, 0.0290, 0.0355, 0.0403, 0.0414, 0.0449, 0.0473, 0.0482, 0.0513]"


>>  a respiratory tract infection


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,a respiratory tract infection,0,28,J98.8,"[J98.8, J06.9, A49.9, J22, J20.9, Z59.3, T17, J04.10, Z13.83, J18.9, P28.9, J39.9, J98.9]","[respiratory tract infection [other specified respiratory disorders], upper respiratory tract infection [acute upper respiratory infection, unspecified], bacterial respiratory infection [bacterial infection, unspecified], acute respiratory infection [unspecified acute lower respiratory infection], bronchial infection [acute bronchitis, unspecified], institution-acquired respiratory infection [problems related to living in residential institution], foreign body in respiratory tract [foreign body in respiratory tract], tracheal infection [acute tracheitis without obstruction], suspected respiratory disease [encounter for screening for respiratory disorder nec], pulmonary infection [pneumonia, unspecified organism], respiratory disease [respiratory condition of newborn, unspecified], disease of upper respiratory tract [disease of upper respiratory tract, unspecified], disease of respiratory system [respiratory disorder, unspecified]]","[0.0248, 0.0507, 0.0675, 0.0737, 0.0751, 0.0786, 0.0834, 0.0868, 0.0906, 0.1005, 0.1033, 0.1024, 0.1031]"


>>  basal cell carcinoma of skin


...

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,severe abdominal pain,0,20,R10.84,"[R10.84, R10.1, R10.9, K38.8, R10.33, R10.2, F45.4, K80.5, R52, M25.51, R14.1, M79.18, R10.30, R10.0, M79.1]","[generalized abdominal pain [generalized abdominal pain], upper abdominal pain [pain localized to upper abdomen], intractable abdominal pain [unspecified abdominal pain], colicky abdominal pain [other specified diseases of appendix], central abdominal pain [periumbilical pain], acute abdominal pain [pelvic and perineal pain], psychosomatic abdominal pain [pain disorders related to psychological factors], recurrent abdominal pain [calculus of bile duct without cholangitis or cholecystitis], heavy pain [pain, unspecified], shoulder pain from abdomen [pain in shoulder], abdominal wind pain [gas pain], abdominal muscle pain [myalgia, other site], site of abdominal pain [lower abdominal pain, unspecified], recurrent acute abdominal pain [acute abdomen], sore pain [myalgia]]","[0.0196, 0.0208, 0.0208, 0.0318, 0.0352, 0.0370, 0.0370, 0.0408, 0.0489, 0.0507, 0.0538, 0.0535, 0.0542, 0.0541, 0.0555]"


>>  squamous cell carcinoma of skin


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,squamous cell carcinoma of skin,0,30,C44.9,"[C44.9, C44.3, D04.9, C44.92, C44.52, C44.99, C44.32, C44.42, C4A.70, C44.320, C44.4, C44.91, C79.9, D04]","[squamous cell carcinoma of skin [other and unspecified malignant neoplasm of skin, unspecified], squamous cell carcinoma of skin of face [other and unspecified malignant neoplasm of skin of other and unspecified parts of face], squamous cell carcinoma in situ of skin [carcinoma in situ of skin, unspecified], scc - squamous cell carcinoma of skin [squamous cell carcinoma of skin, unspecified], squamous cell carcinoma of skin of trunk [squamous cell carcinoma of skin of trunk], mucoepidermoid carcinoma of skin [other specified malignant neoplasm of skin, unspecified], squamous cell carcinoma of skin of and unsp parts of face [squamous cell carcinoma of skin of other and unspecified parts of face], squamous cell carcinoma of skin of neck [squamous cell carcinoma of skin of scalp and neck], trabecular cell carcinoma of skin [merkel cell carcinoma of unspecified lower limb, including hip], squamous cell carcinoma of skin of unspecified parts of face [squamous cell carcinoma of skin of unspecified parts of face], squamous cell carcinoma of scalp [other and unspecified malignant neoplasm of skin of scalp and neck], basal cell carcinoma of skin [basal cell carcinoma of skin, unspecified], squamous cell carcinomatosis [secondary malignant neoplasm of unspecified site], carcinoma in situ of skin [carcinoma in situ of skin]]","[0.0000, 0.0244, 0.0289, 0.0333, 0.0337, 0.0420, 0.0519, 0.0564, 0.0600, 0.0624, 0.0623, 0.0620, 0.0638, 0.0639]"


>>  type 2 diabetes mellitus


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,type 2 diabetes mellitus,0,23,E11,"[E11, E11.9, E11.8, E11.3, O24.11, E13.9, E11.43, E10.9, E11.69, E11.4, E11.2, E11.51, E11.62, E11.44, Z86.3]","[type 2 diabetes mellitus [type 2 diabetes mellitus], diabetes mellitus type 2 [type 2 diabetes mellitus without complications], disorder associated with type 2 diabetes mellitus [type 2 diabetes mellitus with unspecified complications], disorder of eye with type 2 diabetes mellitus [type 2 diabetes mellitus with ophthalmic complications], pre-existing type 2 diabetes mellitus [pre-existing type 2 diabetes mellitus, in pregnancy], secondary diabetes mellitus [other specified diabetes mellitus without complications], neurological disorder with type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic autonomic (poly)neuropathy], diabetes mellitus [type 1 diabetes mellitus without complications], diabetic dyslipidemia associated with type 2 diabetes mellitus [type 2 diabetes mellitus with other specified complication], neurological disorder with diabetes type 2 [type 2 diabetes mellitus with neurological complications], kidney disorder associated with type 2 diabetes mellitus [type 2 diabetes mellitus with kidney complications], peripheral circulatory disorder associated with type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic peripheral angiopathy without gangrene], type 2 diabetes mellitus with skin complications [type 2 diabetes mellitus with skin complications], disorder of nervous system due to type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic amyotrophy], history of diabetes mellitus type 2 (situation) [personal history of endocrine, nutritional and metabolic diseases]]","[0.0000, 0.0134, 0.0212, 0.0357, 0.0360, 0.0397, 0.0455, 0.0462, 0.0499, 0.0523, 0.0537, 0.0548, 0.0549, 0.0546, 0.0554]"


In [ ]:
for chunk in snomed_chunks:

    print ('>> ',chunk)
    display(get_codes (snomed_lp, chunk, vocab='snomed_code', aux_label=True))

>>  down syndrome


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,down syndrome,0,12,408338009,"[408338009, 25088000, 45056002, 20741006, 29696001, 255518004, 244840000, 49764008, 245777007, 702851008, 125490008, 60736004, 245780008, 37822005, 255521002, 27033000, 48077000, 25659002, 272168005, 246199000, 263743007, 129453003]","[suspected downs syndrome, dimpled structure, abnormal shortened structure, dropsy, downward displacement, downward, suppressive, down-regulation, lower lip, down syndrome clinic, decreased elasticity, lookdown, suppressed, lower back structure, downward gaze, lower abdomen, lower jaw, narrowed structure, droppings, deficiency, downsloping, deficiency of]","[Context-dependent, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Qualifier Value, Qualifier Value, Observable Entity, Body Structure, Location, Morph Abnormality, Organism, Qualifier Value, Body Structure, Qualifier Value, Body Structure, Body Structure, Morph Abnormality, Substance, Attribute, Qualifier Value, Attribute]","[0.0790, 0.1054, 0.1059, 0.1066, 0.1068, 0.1114, 0.1157, 0.1159, 0.1179, 0.1267, 0.1238, 0.1247, 0.1241, 0.1257, 0.1252, 0.1274, 0.1288, 0.1254, 0.1270, 0.1273, 0.1288, 0.1267]"


>>  adenocarcinoma


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,adenocarcinoma,0,13,72495009,"[72495009, 58069009, 450948005, 45410002, 32968003, 1187332001, 302827006, 86293007, 28558000, 89740008, 79143006, 51642000, 1259176001, 64000002, 4584002, 90725004, 1259175002, 115215004, 21008007, 900006]","[mucous adenocarcinoma, ceruminous adenocarcinoma, serrated adenocarcinoma, acinar adenocarcinoma, inflammatory adenocarcinoma, adenocarcinoma (morphologic abnormality), adenocystic carcinoma, adenocarcinoid tumor, villous adenocarcinoma, lobular adenocarcinoma, pseudomucinous adenocarcinoma, adenocarcinoma in situ, adenocarcinoma of head, adnexal carcinoma, scirrhous adenocarcinoma, serous adenocarcinoma, adenocarcinoma of neck, adenoma and/or adenocarcinoma, cystadenocarcinoma, mucin-producing adenocarcinoma]","[Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, No_Concept_Class, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, No_Concept_Class, Morph Abnormality, Morph Abnormality, Morph Abnormality, No_Concept_Class, Morph Abnormality, Morph Abnormality, Morph Abnormality]","[0.0432, 0.0496, 0.0513, 0.0520, 0.0563, 0.0578, 0.0594, 0.0615, 0.0601, 0.0617, 0.0708, 0.0683, 0.0697, 0.0699, 0.0734, 0.0720, 0.0731, 0.0772, 0.0792, 0.0801]"


>>  aortic valve stenosis


...

,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,squamous cell carcinoma,0,22,1162767002,"[1162767002, 37304002, 89906000, 85956000, 118286007, 39056008, 259641008, 399739006, 733897002, 394961009, 90725004, 59367005, 10288008, 65692009, 72495009, 12478003, 1162893000]","[squamous carcinoma, basisquamous cell carcinoma, verrucous squamous cell carcinoma, adenoid squamous cell carcinoma, squamous cell neoplasm, papillary squamous cell carcinoma, squamous cell carcinoma antigen, carcinosarcomatous squamous cell carcinoma, pseudovascular squamous cell carcinoma, squamous cell carcinoma antigen level, serous carcinoma, mixed adenocarcinoma and squamous cell carcinoma, squamous cell carcinoma, spindle cell, polypoid squamous cell carcinoma, mucous carcinoma, microinvasive squamous cell carcinoma, squamous cell carcinoma in situ (morphologic abnormality)]","[No_Concept_Class, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Substance, Morph Abnormality, Morph Abnormality, Procedure, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, No_Concept_Class]","[0.0071, 0.0348, 0.0455, 0.0504, 0.0601, 0.0610, 0.0625, 0.0629, 0.0692, 0.0707, 0.0741, 0.0764, 0.0775, 0.0799, 0.0819, 0.0818, 0.0824]"


>>  stage IIIB


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,stage IIIB,0,9,258219007,"[258219007, 307204003, 258224005, 307203009, 258352004, 722741009, 7348004, 1222805004, 258195006, 398428002, 1222802001, 316895001, 719235001, 83326004, 1222766008, 1222813003, 446852007, 30417003, 1222804000, 38714005, 90676008, 1222803006, 1222806003, 1222807007, 20365006]","[stage ii, grade iiib, stage iii, grade iiia, grade ii, cdc group iib, factor ii, iiib, type ii, s. iiib, iiia, compound grade iiic, cdc group iic, cdc group iif, iia, iii:2, iiwis, htlv-ii, iiia2, igf-ii, stlv-iii, iiia1, iiic, iiic1, vin iii]","[Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Organism, Substance, No_Concept_Class, Qualifier Value, Organism, No_Concept_Class, Qualifier Value, Organism, Organism, No_Concept_Class, No_Concept_Class, Organism, Organism, Qualifier Value, Substance, Organism, No_Concept_Class, No_Concept_Class, No_Concept_Class, Morph Abnormality]","[0.0370, 0.0478, 0.0715, 0.0736, 0.0763, 0.0808, 0.0817, 0.0800, 0.0821, 0.0815, 0.0824, 0.0856, 0.0874, 0.0897, 0.0876, 0.0905, 0.0887, 0.0911, 0.0909, 0.0930, 0.0945, 0.0949, 0.0935, 0.0974, 0.0935]"


>>  mediastinal lymph nodes


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,mediastinal lymph nodes,0,22,245276004,"[245276004, 181760002, 62683002, 178297005, 260926004, 281779003, 364445008, 127931000, 734421006, 198237001, 72410000, 258311008, 279814004, 245344006, 198080002, 178329003, 279818001, 234272006, 279813005, 196242000, 27416003, 31808004]","[mediastinal lymph nodes, mediastinal lymph node, mediastinal lymph node structure, mediastinal lymph nodes sampling, mediastinal node status, mediastinal lymphatic structure, mediastinal feature, retrotracheal lymph node (mediastinal), specimen from mass of mediastinal lymph node, mediastinal vein, mediastinal structure, mediastinal lymph node tumour involvement, pulmonary lymph nodes, perigastric lymph node, mediastinal artery, mediastinal lymphangiogram, paratracheal lymph nodes, biopsy of mediastinal lymph node, diaphragmatic lymph nodes, lymphatic vessel of thymus and mediastinum, mediastinal pleura, intrapulmonary lymph node]","[Body Structure, Body Structure, Body Structure, Procedure, Attribute, Body Structure, Observable Entity, Body Structure, Specimen, Body Structure, Body Structure, Staging / Scales, Body Structure, Body Structure, Body Structure, Procedure, Body Structure, Procedure, Body Structure, Body Structure, Body Structure, Body Structure]","[0.0000, 0.0226, 0.0342, 0.0511, 0.0635, 0.0670, 0.0777, 0.0813, 0.0892, 0.0882, 0.0875, 0.0964, 0.0986, 0.1019, 0.1013, 0.1067, 0.1059, 0.1080, 0.1085, 0.1092, 0.1049, 0.1086]"


In [ ]:
clinical_chunks = ['bladder cancer',
 'anemia in chronic kidney disease',
 'castleman disease',
 'congestive heart failure',
 'diabetes mellitus type 2',
 'lymph nodes of multiple sites',
 'malignant melanoma of skin',
 'malignant neoplasm of lower lobe, bronchus',
 'metastatic lung cancer',
 'secondary malignant neoplasm of bone',
 'type 2 diabetes mellitus',
 'type 2 diabetes mellitus/insulin',
 'unsp malignant neoplasm of lymph node']


for chunk in clinical_chunks:

    print ('>> ',chunk)

    print ('icd10cm_code')
    display(get_codes (hcc_lp, chunk, vocab='icd10cm_hcc_code', hcc=True))

    print ('snomed_code')
    display(get_codes (snomed_lp, chunk, vocab='snomed_code', aux_label=True))

>>  bladder cancer
icd10cm_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,bladder cancer,0,13,C67.9,"[C67.9, Z12.6, D09.0, D49.4, C79.11]","[bladder cancer [malignant neoplasm of bladder, unspecified], suspected bladder cancer [encounter for screening for malignant neoplasm of bladder], cancer in situ of urinary bladder [carcinoma in situ of bladder], tumor of bladder neck [neoplasm of unspecified behavior of bladder], malignant tumour of bladder neck [secondary malignant neoplasm of bladder]]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]","[1, 1, 1, 1, 1]","[1, 0, 0, 0, 1]","[11, 0, 0, 0, 8]"


snomed_code


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,bladder cancer,0,13,1259400001,"[1259400001, 315269000, 1259401002, 1237457005, 1263639008, 27090000, 1237465008, 1259402009, 1208460000, 53530009, 1259705005, 12400006, 315271000, 309273004]","[primary carcinoma of bladder, suspected bladder cancer, primary transitional cell carcinoma of bladder, secondary carcinoma of bladder, secondary transitional cell carcinoma of bladder, urothelial carcinoma, secondary adenocarcinoma of bladder, primary adenocarcinoma of bladder, sarcomatoid urothelial carcinoma of urinary bladder, urothelial carcinoma in situ, primary sarcomatoid urothelial carcinoma of urinary bladder, papillary urothelial carcinoma, suspected urothelial cancer, bladder biopsy specimen]","[No_Concept_Class, Context-dependent, No_Concept_Class, No_Concept_Class, No_Concept_Class, Morph Abnormality, No_Concept_Class, No_Concept_Class, No_Concept_Class, Morph Abnormality, No_Concept_Class, Morph Abnormality, Context-dependent, Specimen]","[0.0790, 0.0904, 0.0985, 0.1035, 0.1039, 0.1091, 0.1118, 0.1218, 0.1292, 0.1429, 0.1442, 0.1513, 0.1547, 0.1527]"


>>  anemia in chronic kidney disease
icd10cm_code


...

,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,type 2 diabetes mellitus/insulin,0,31,161445009,"[161445009, 160303001, 1251386008, 472969004, 473127005, 1259048007, 1172389004, 772788006, 405751000, 472970003, 721121000124104, 1217044000, 1196922005, 704145004, 444913002, 1196923000, 1217674007]","[h/o: diabetes mellitus, fh: diabetes mellitus, disorder of skin due to diabetes mellitus, history of diabetes mellitus type ii, suspected diabetes mellitus, brachial plexus lesion due to diabetes mellitus, disorder due to malnutrition related diabetes mellitus, assessment of risk of type 2 diabetes mellitus, diabetes type, history of diabetes mellitus type i, diabetes mellitus type 1 stage 2, ketosis-resistant diabetes mellitus, neovascular glaucoma due to diabetes mellitus type 2, maternal history of insulin dependent diabetes mellitus, diabetes mellitus service, neovascular glaucoma due to type i diabetes mellitus, neurotrophic keratitis due to diabetes mellitus]","[Context-dependent, Context-dependent, No_Concept_Class, Context-dependent, Context-dependent, No_Concept_Class, No_Concept_Class, Procedure, Observable Entity, Context-dependent, No_Concept_Class, No_Concept_Class, No_Concept_Class, Context-dependent, Qualifier Value, No_Concept_Class, No_Concept_Class]","[0.0800, 0.0853, 0.0924, 0.1010, 0.1038, 0.1109, 0.1116, 0.1212, 0.1204, 0.1199, 0.1295, 0.1300, 0.1376, 0.1336, 0.1408, 0.1390, 0.1467]"


>>  unsp malignant neoplasm of lymph node
icd10cm_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,unsp malignant neoplasm of lymph node,0,36,C77.9,"[C77.9, C77, D36.0, C77.2, C77.1, C77.8, C77.5, D49.2, C77.0, C96.9]","[secondary and unsp malignant neoplasm of lymph node, unsp [secondary and unspecified malignant neoplasm of lymph node, unspecified], secondary and unspecified malignant neoplasm of lymph nodes [secondary and unspecified malignant neoplasm of lymph nodes], benign neoplasm of lymph node [benign neoplasm of lymph nodes], secondary and unsp malignant neoplasm of intra-abd nodes [secondary and unspecified malignant neoplasm of intra-abdominal lymph nodes], secondary and unspecified malignant neoplasm of intrathoracic lymph nodes [secondary and unspecified malignant neoplasm of intrathoracic lymph nodes], secondary and unspecified malignant neoplasm of lymph nodes of multiple regions [secondary and unspecified malignant neoplasm of lymph nodes of multiple regions], secondary and unsp malignant neoplasm of intrapelv nodes [secondary and unspecified malignant neoplasm of intrapelvic lymph nodes], neoplasm of lymph node [neoplasm of unspecified behavior of bone, soft tissue, and skin], secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck [secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck], malignant neoplasm of lymphoid, hematopoietic and related tissue, unspecified [malignant neoplasm of lymphoid, hematopoietic and related tissue, unspecified]]","[0.0606, 0.0681, 0.0754, 0.0920, 0.0980, 0.0989, 0.1034, 0.0968, 0.1051, 0.1055]","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 0, 0, 1, 1, 1, 1, 0, 1, 1]","[10, 0, 0, 8, 8, 8, 8, 0, 8, 10]"


snomed_code


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,unsp malignant neoplasm of lymph node,0,36,733036009,"[733036009, 1240414004, 1172695008, 1172592001, 55150002, 189958005, 129142003, 67824002, 414628006, 789471007, 1255723009, 86251006, 1187127006, 400110009, 1269006002, 64626006, 1196815001]","[non-sebaceous lymphadenoma (morphologic abnormality), unclassified malignant tumor, noncleaved malignant lymphoma, malignant lymphoma, non-hodgkin's, malignant lymphoma, nodular, nos, lymphomatous tumour, benign, benign lymphoepithelial lesion (morphologic abnormality), lymphosarcoma, nos, lymphoid neoplasm, follicular neoplasm of uncertain malignant potential, benign neoplasm of peripheral nerve of head and neck, neoplasm, nos, benign neuroendocrine neoplasm (morphologic abnormality), benign blood vessel neoplasm, noninvasive premalignant neoplasm (morphologic abnormality), lymphoid nodule, benign neoplasm of lacrimal apparatus]","[Morph Abnormality, No_Concept_Class, No_Concept_Class, No_Concept_Class, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, No_Concept_Class, Morph Abnormality, No_Concept_Class, Morph Abnormality, No_Concept_Class, Body Structure, No_Concept_Class]","[0.1023, 0.1306, 0.1372, 0.1388, 0.1394, 0.1393, 0.1370, 0.1531, 0.1545, 0.1610, 0.1590, 0.1598, 0.1621, 0.1648, 0.1733, 0.1729, 0.1783]"


##  How to integrate resolvers with NER models in the same pipeline


In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

clinical_ner = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = medical.NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['PROBLEM'])

c2doc = nlp.Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

icd10_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")

sbert_resolver_pipeline = nlp.Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10_resolver])

data_ner = spark.createDataFrame([[""]]).toDF("text")

sbert_models = sbert_resolver_pipeline.fit(data_ner)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
[OK!]


In [ ]:
clinical_note = 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for starvation ketosis, as she reported poor oral intake for three days prior to admission. However, serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.'

print (clinical_note)

clinical_note_df = spark.createDataFrame([[clinical_note]]).toDF("text")


A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20

In [ ]:
icd10_result = sbert_models.transform(clinical_note_df)

In [ ]:
import pyspark.sql.functions as F

pd.set_option('display.max_colwidth', 0)

def get_icd_codes(icd10_res):

    icd10_df = icd10_res.select(F.explode(F.arrays_zip(icd10_res.ner_chunk.result,
                                                       icd10_res.ner_chunk.metadata,
                                                       icd10_res.icd10cm_code.result,
                                                       icd10_res.icd10cm_code.metadata)).alias("cols")) \
                                 .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                         F.expr("cols['0']").alias("ner_chunk"),
                                         F.expr("cols['1']['entity']").alias("entity"),
                                         F.expr("cols['2']").alias("icd10_code"),
                                         F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                         F.expr("cols['3']['all_k_resolutions']").alias("resolutions")).toPandas()



    codes = []
    resolutions = []

    for code, resolution in zip(icd10_df['all_codes'], icd10_df['resolutions']):

        codes.append(code.split(':::'))
        resolutions.append(resolution.split(':::'))

    icd10_df['all_codes'] = codes
    icd10_df['resolutions'] = resolutions

    return icd10_df

In [ ]:
%%time

res_pd = get_icd_codes(icd10_result)

CPU times: user 632 ms, sys: 101 ms, total: 733 ms
Wall time: 1min 13s


In [ ]:
res_pd.head(15)

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions
0,0,gestational diabetes mellitus,PROBLEM,O24.4,"[O24.4, O24.41, O24.43, Z86.32, Z87.5, O24.31, O24.11, O24.1, O24.81]","[gestational diabetes mellitus [gestational diabetes mellitus], gestational diabetes mellitus (disorder) [gestational diabetes mellitus in pregnancy], postpartum gestational diabetes mellitus [gestational diabetes mellitus in the puerperium], history of gestational diabetes mellitus [personal history of gestational diabetes], history of gestational diabetes mellitus (situation) [personal history of complications of pregnancy, childbirth and the puerperium], pre-existing diabetes mellitus in pregnancy [unspecified pre-existing diabetes mellitus in pregnancy], pre-existing diabetes mellitus in pregnancy (disorder) [pre-existing type 2 diabetes mellitus, in pregnancy], pre-existing type 2 diabetes mellitus in pregnancy [pre-existing type 2 diabetes mellitus, in pregnancy, childbirth and the puerperium], other pre-existing diabetes mellitus in pregnancy [other pre-existing diabetes mellitus in pregnancy]]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O24.11,"[O24.11, E11.8, E11, E13.9, E11.9, E11.3, E11.44, Z86.3, Z86.39, E11.32, E11.43, E11.5]","[pre-existing type 2 diabetes mellitus [pre-existing type 2 diabetes mellitus, in pregnancy], disorder associated with type 2 diabetes mellitus [type 2 diabetes mellitus with unspecified complications], type 2 diabetes mellitus [type 2 diabetes mellitus], secondary diabetes mellitus [other specified diabetes mellitus without complications], diabetes mellitus type 2 [type 2 diabetes mellitus without complications], disorder of eye due to type 2 diabetes mellitus [type 2 diabetes mellitus with ophthalmic complications], disorder of nervous system due to type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic amyotrophy], history of diabetes mellitus type 2 (situation) [personal history of endocrine, nutritional and metabolic diseases], history of diabetes mellitus type 2 [personal history of other endocrine, nutritional and metabolic disease], secondary endocrine diabetes mellitus [type 2 diabetes mellitus with mild nonproliferative diabetic retinopathy], neurological disorder with type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic autonomic (poly)neuropathy], peripheral circulatory disorder due to type 2 diabetes mellitus [type 2 diabetes mellitus with circulatory complications]]"
2,0,T2DM,PROBLEM,P70.2,"[P70.2, C80.1, G12.1, Q85.0, C77.9, C50.9, C43.9, E72.3, C57.00, Z72.81, C79.49, E75.0, C76.8, T80.8]","[tndm2 [neonatal diabetes mellitus], t2 category [malignant (primary) neoplasm, unspecified], sma2 [other inherited spinal muscular atrophy], nf2 [neurofibromatosis (nonmalignant)], pn2 category [secondary and unspecified malignant neoplasm of lymph node, unspecified], pn2a category [malignant neoplasm of breast of unspecified site], pn2c category [malignant melanoma of skin, unspecified], ga ii [disorders of lysine and hydroxylysine metabolism], pt2c category (finding) [malignant neoplasm of unspecified fallopian tube], truancy [antisocial behavior], pt2c category [secondary malignant neoplasm of other parts of nervous system], tsd [gm2 gangliosidosis], pn2c category (finding) [malignant neoplasm of other specified ill-defined sites], trali [other complications following infusion, transfusion and therapeutic injection]]"
3,0,HTG-induced pancreatitis,PROBLEM,K85.20,"[K85.20, K85.9, K85.3, K85.90, K85, K86.0, K85.8, F10.2, K85.91, K85.2, K86.1, K91.8, K85.80, K85.92]","[alcohol-induced pancreatitis [alcohol induced acute pancreatitis without necrosis or infection], pancreatitis [acute pancreatitis, unspecified], drug induced acute pancreatitis [drug induced acute pancreatitis], ap - acute pancreatitis [acute pancreatitis without necrosis or infection, unspecified], acute pancreatitis [acute pancreatitis], alcohol-induced chronic pancreatitis [alcohol-induced chronic pancreatitis], aut

Lets apply some HTML formating by using `sparknlp_display` library to see the results of the pipeline in a nicer layout:

In [ ]:
# with light pipeline
light_model = nlp.LightPipeline(sbert_models)

vis = nlp.viz.EntityResolverVisualizer()

# Change color of an entity label
vis.set_label_colors({'PROBLEM':'#008080'})

light_data_icd = light_model.fullAnnotate(clinical_note)

vis.display(light_data_icd[0], 'ner_chunk', 'icd10cm_code')


## BertSentenceChunkEmbeddings

This annotator let users to aggregate sentence embeddings and ner chunk embeddings to get more specific and accurate resolution codes. It works by averaging context and chunk embeddings to get contextual information. Input to this annotator is the context (sentence) and ner chunks, while the output is embedding for each chunk that can be fed to the resolver model. The `setChunkWeight` parameter can be used to control the influence of surrounding context.

For more information and examples of `BertSentenceChunkEmbeddings` annotator, you can check here:
[24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb)

### ICD10CM with BertSentenceChunkEmbeddings

Lets do the same process by using `BertSentenceEmbeddings` annotator and compare the results. We will create a new pipeline by using this annotator with SentenceEntityResolverModel.

In [ ]:
#Get average sentence-chunk Bert embeddings
sentence_chunk_embeddings = medical.BertSentenceChunkEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence", "ner_chunk"])\
    .setOutputCol("sbert_embeddings")\
    .setCaseSensitive(False)\
    .setChunkWeight(0.5) #default : 0.5


icd10_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")

resolver_pipeline_SCE = nlp.Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        sentence_chunk_embeddings,
        icd10_resolver])


empty_data = spark.createDataFrame([['']]).toDF("text")
model_SCE = resolver_pipeline_SCE.fit(empty_data)

sbiobert_base_cased_mli download started this may take some time.
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
[OK!]


In [ ]:
icd10_result_SCE = model_SCE.transform(clinical_note_df)

In [ ]:
%%time

res_SCE_pd = get_icd_codes(icd10_result_SCE)

CPU times: user 180 ms, sys: 29.2 ms, total: 209 ms
Wall time: 32.3 s


In [ ]:
res_SCE_pd.head(15)

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions
0,0,gestational diabetes mellitus,PROBLEM,O24.4,"[O24.4, O24.43, O24.41, O24.434, O24.42, O24.414, O24.13]","[gestational diabetes mellitus, class d [gestational diabetes mellitus], gestational diabetes mellitus in the puerperium [gestational diabetes mellitus in the puerperium], gestational diabetes mellitus, class a>1< [gestational diabetes mellitus in pregnancy], gestational diabetes mellitus in the puerperium, insulin controlled [gestational diabetes mellitus in the puerperium, insulin controlled], gestational diabetes mellitus in childbirth [gestational diabetes mellitus in childbirth], gestational diabetes mellitus in pregnancy, insulin controlled [gestational diabetes mellitus in pregnancy, insulin controlled], pre-existing type 2 diabetes mellitus, in the puerperium [pre-existing type 2 diabetes mellitus, in the puerperium]]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O24.13,"[O24.13, E11.6, E11.43, O24.11, E11.10, E11.33, E11.4, O24.1, O24.12, E11.1, K31.8, E11.69, E11.3, O24.4, E11.2, E10.9, E11.0]","[pre-existing type 2 diabetes mellitus, in the puerperium [pre-existing type 2 diabetes mellitus, in the puerperium], gastroparesis co-occurrent and due to type 2 diabetes mellitus [type 2 diabetes mellitus with other specified complications], gastroparesis with type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic autonomic (poly)neuropathy], pregnancy and type 2 diabetes mellitus [pre-existing type 2 diabetes mellitus, in pregnancy], ketoacidosis due to type 2 diabetes mellitus [type 2 diabetes mellitus with ketoacidosis without coma], moderate nonproliferative retinopathy co-occurrent and due to type 2 diabetes mellitus [type 2 diabetes mellitus with moderate nonproliferative diabetic retinopathy], cranial nerve palsy co-occurrent and due to type 2 diabetes mellitus [type 2 diabetes mellitus with neurological complications], pre-existing type 2 diabetes mellitus in pregnancy [pre-existing type 2 diabetes mellitus, in pregnancy, childbirth and the puerperium], pre-existing type 2 diabetes mellitus, in childbirth [pre-existing type 2 diabetes mellitus, in childbirth], ketoacidosis due to type 2 diabetes mellitus (disorder) [type 2 diabetes mellitus with ketoacidosis], diabetic gastroparesis associated with type 2 diabetes mellitus [other specified diseases of stomach and duodenum], angina due to type 2 diabetes mellitus [type 2 diabetes mellitus with other specified complication], rubeosis iridis co-occurrent and due to type 2 diabetes mellitus [type 2 diabetes mellitus with ophthalmic complications], gestational diabetes mellitus, class d [gestational diabetes mellitus], persistent microalbuminuria due to type 2 diabetes mellitus [type 2 diabetes mellitus with kidney complications], acanthosis nigricans due to type 2 diabetes mellitus [type 1 diabetes mellitus without complications], ketoacidotic coma due to type 2 diabetes mellitus [type 2 diabetes mellitus with hyperosmolarity]]"
2,0,T2DM,PROBLEM,R70.1,"[R70.1, S32.021G, E16.1, E11.37X2, P70.2, T45.2X, S22.021G, L65.8, E11.37X1, K91.2, Z37.9, E89.1, I87.023, O42.912, I87.093, E72.8]","[reticulcytosis after b12 [abnormal plasma viscosity], stable burst fx second lum vert, subs for fx w delay heal [stable burst fracture of second lumbar vertebra, subsequent encounter for fracture with delayed healing], ectopic igf-2 hypoglycemia (disorder) [other hypoglycemia], type 2 diab with diab mclr edema, resolved fol trtmt, l eye [type 2 diabetes mellitus with diabetic macular edema, resolved following treatment, left eye], diabetes mellitus, transient neonatal 2 (disorder) [neonatal diabetes mellitus], reticulcytosis after b12 (finding) [poisoning by, adverse effect of and underdosing of vitamins], stable burst fx second thor vert, subs for fx w delay heal [stable burst fracture of second thoracic vertebra, subsequent encounter for fracture with delayed healing], postinfectional hypotrichosis [other sp

In [ ]:
icd10_SCE_lp = nlp.LightPipeline(model_SCE)

light_result = icd10_SCE_lp.fullAnnotate(clinical_note)

visualiser = nlp.viz.EntityResolverVisualizer()

# Change color of an entity label
visualiser.set_label_colors({'PROBLEM':'#008080'})

visualiser.display(light_result[0], 'ner_chunk', 'icd10cm_code')

**Lets compare the results that we got from these two methods.**

In [ ]:
sentence_df = icd10_result.select(F.explode(F.arrays_zip(icd10_result.sentence.metadata,
                                                         icd10_result.sentence.result)).alias("cols")) \
                                .select( F.expr("cols['0']['sentence']").alias("sent_id"),
                                         F.expr("cols['1']").alias("sentence_all")).toPandas()

comparison_df = pd.merge(res_pd.loc[:,'sent_id':'resolutions'],res_SCE_pd.loc[:,'sent_id':'resolutions'], on=['sent_id',"ner_chunk", "entity"], how='inner')
comparison_df.columns=['sent_id','ner_chunk', 'entity', 'icd10_code', 'all_codes', 'resolutions', 'icd10_code_SCE', 'all_codes_SCE', 'resolutions_SCE']

comparison_df = pd.merge(sentence_df, comparison_df,on="sent_id").drop('sent_id', axis=1)
comparison_df.head(15)

,sentence_all,ner_chunk,entity,icd10_code,all_codes,resolutions,icd10_code_SCE,all_codes_SCE,resolutions_SCE
0,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",gestational diabetes mellitus,PROBLEM,O24.4,"[O24.4, O24.41, O24.43, Z86.32, Z87.5, O24.31, O24.11, O24.1, O24.81]","[gestational diabetes mellitus [gestational diabetes mellitus], gestational diabetes mellitus (disorder) [gestational diabetes mellitus in pregnancy], postpartum gestational diabetes mellitus [gestational diabetes mellitus in the puerperium], history of gestational diabetes mellitus [personal history of gestational diabetes], history of gestational diabetes mellitus (situation) [personal history of complications of pregnancy, childbirth and the puerperium], pre-existing diabetes mellitus in pregnancy [unspecified pre-existing diabetes mellitus in pregnancy], pre-existing diabetes mellitus in pregnancy (disorder) [pre-existing type 2 diabetes mellitus, in pregnancy], pre-existing type 2 diabetes mellitus in pregnancy [pre-existing type 2 diabetes mellitus, in pregnancy, childbirth and the puerperium], other pre-existing diabetes mellitus in pregnancy [other pre-existing diabetes mellitus in pregnancy]]",O24.4,"[O24.4, O24.43, O24.41, O24.434, O24.42, O24.414, O24.13]","[gestational diabetes mellitus, class d [gestational diabetes mellitus], gestational diabetes mellitus in the puerperium [gestational diabetes mellitus in the puerperium], gestational diabetes mellitus, class a>1< [gestational diabetes mellitus in pregnancy], gestational diabetes mellitus in the puerperium, insulin controlled [gestational diabetes mellitus in the puerperium, insulin controlled], gestational diabetes mellitus in childbirth [gestational diabetes mellitus in childbirth], gestational diabetes mellitus in pregnancy, insulin controlled [gestational diabetes mellitus in pregnancy, insulin controlled], pre-existing type 2 diabetes mellitus, in the puerperium [pre-existing type 2 diabetes mellitus, in the puerperium]]"
1,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",subsequent type two diabetes mellitus,PROBLEM,O24.11,"[O24.11, E11.8, E11, E13.9, E11.9, E11.3, E11.44, Z86.3, Z86.39, E11.32, E11.43, E11.5]","[pre-existing type 2 diabetes mellitus [pre-existing type 2 diabetes mellitus, in pregnancy], disorder associated with type 2 diabetes mellitus [type 2 diabetes mellitus with unspecified complications], type 2 diabetes mellitus [type 2 diabetes mellitus], secondary diabetes mellitus [other specified diabetes mellitus without complications], diabetes mellitus type 2 [type 2 diabetes mellitus without complications], disorder of eye due to type 2 diabetes mellitus [type 2 diabetes mellitus with ophthalmic complications], disorder of nervous system due to type 2 diabetes mellitus [type 2 diabetes mellitus with diabetic amyotrophy], history of diabetes mellitus type 2 (situation) [personal history of endocrine, nutritional and metabolic diseases], history of diabetes mellitus type 2 [personal history of other endocrine, nutritional and metabolic disease], secondary endocrine diabetes mellitus [type 2 diabetes mellitus with mild nonproliferative diabetic retinopathy], neurological disorder with type 2 diabetes mellitus [type 2 diabetes mellitus